---
* *** 원본 소스는 홍콩과기대 김성훈 교수님의  [Github link](https://github.com/hunkim/DeepLearningZeroToAll)를 참조하세요 ***     
* *** Watson studio notebook 에서 작업 가능하도록 수정하였으며***
* *** 한글로 설명된 부분은 제가 스터디 하면서 이해한 내용을 추가한 것입니다. ***
---

## Lab-02-2 linear regression feed

* ***바로 앞에 Lab 2-1 하고 처리하는 로직은 동일합니다 ***
* ***단, placeholder 라는 함수를 사용해서  input value 를 변수처리를 하도록 했고 ***
* ***앞에서 얘기했듯이 이제 예측을 한번 해보는 겁니다.***

In [1]:
import tensorflow as tf
tf.set_random_seed(777)  # for reprducibilty

* ***W,b 에 이름을 부여하고, 시작 값을 random으로 생성합니다.***

In [2]:
# Try to find value for W and b to compute y_data = x_data * W + b  
# We know that W should be 1 and b should be 0
# But let's TensorFlow figure it out 
W = tf.Variable(tf.random_normal([1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

* ***값이 어떻게 들어올지 모르니, 행렬의 크기를 None, 즉 모른다로 정의합니다.***

In [3]:
#### Now we can use X and Y in place of x_data and y_data
#### placeholders for a tensor that will be always fed using feed_dict
#### See http://stackoverflow.com/questions/36693740/
X = tf.placeholder(tf.float32, shape=[None])
Y = tf.placeholder(tf.float32, shape=[None])

* ***앞에 Lab2-1과 동일합니다.***

In [4]:
# Out hypothesis XW+b
hypothesis = X * W + b

# cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [5]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

In [6]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

* ***Lab2-1과 동일하지만, run 할때 training data를 받습니다.***

In [7]:
for step in range(2001):
    cost_val, W_val, b_val, _ =  sess.run([cost, W, b, train], feed_dict={X: [1, 2, 3], Y: [1, 2, 3]})
    if step % 200 == 0:
        print(step, cost_val, W_val, b_val)


0 3.52408 [ 2.12867713] [-0.85235667]
200 0.0703044 [ 1.30721486] [-0.69837117]
400 0.0268456 [ 1.18983996] [-0.4315508]
600 0.010251 [ 1.11730957] [-0.26667204]
800 0.00391432 [ 1.07249022] [-0.16478711]
1000 0.00149468 [ 1.04479456] [-0.10182849]
1200 0.00057074 [ 1.02768016] [-0.06292368]
1400 0.000217935 [ 1.01710474] [-0.03888312]
1600 8.32203e-05 [ 1.01056981] [-0.02402747]
1800 3.17767e-05 [ 1.00653136] [-0.01484741]
2000 1.21343e-05 [ 1.00403607] [-0.00917497]


##### 결과는 1,0 으로 수렴하지요. 

###  자 ! 이제 우리의 모델을 가지고 시험(예측)을 해보겠습니다.

In [8]:
print(sess.run(hypothesis, feed_dict={X: [5]}))
print(sess.run(hypothesis, feed_dict={X: [2.5]}))
print(sess.run(hypothesis, feed_dict={X: [1.5, 3.5]}))

[ 5.0110054]
[ 2.50091505]
[ 1.49687922  3.50495124]


* ***주어진 x 값과 동일한 y 값이 나와야지요***
* ***여기서 오해하면 안되는 것이,,, W,b 가 근사치로 정해졌기 때문에  예측치도 정확하게 일치 하지는 않습니다.***

### 자 ! 이제 새로운 training 데이터를 가지고, 손쉽게  새로운 모델을 만들어 보겠습니다.

In [9]:
for step in range(2001):
    cost_val, W_val, b_val, _ = \
        sess.run([cost, W, b, train],
                 feed_dict={X: [1, 2, 3, 4, 5],
                            Y: [2.1, 3.1, 4.1, 5.1, 6.1]})
    if step % 200 == 0:
        print(step, cost_val, W_val, b_val)

0 1.20359 [ 1.06969857] [ 0.01276637]
200 0.0499406 [ 1.14459527] [ 0.5779649]
400 0.0128862 [ 1.07344985] [ 0.83482295]
600 0.00332502 [ 1.03730989] [ 0.96529907]
800 0.000857956 [ 1.01895225] [ 1.03157616]
1000 0.000221378 [ 1.00962698] [ 1.06524324]
1200 5.71206e-05 [ 1.00489008] [ 1.08234489]
1400 1.47405e-05 [ 1.0024842] [ 1.09103119]
1600 3.80434e-06 [ 1.00126207] [ 1.09544384]
1800 9.81556e-07 [ 1.00064111] [ 1.09768546]
2000 2.5373e-07 [ 1.00032604] [ 1.09882331]


* ***새로운 W,b가 구해졌나요 ? ***
* ***다시 테스트 (예측) 해봅시다.***

In [10]:
print(sess.run(hypothesis, feed_dict={X: [5]}))
print(sess.run(hypothesis, feed_dict={X: [2.5]}))
print(sess.run(hypothesis, feed_dict={X: [1.5, 3.5]}))

[ 6.10045338]
[ 3.59963846]
[ 2.59931231  4.59996414]


### 지금 두개의 모델을 만들었습니다. 만약 이 모델을  저장, 관리하고,  다른 사용자들에게 공유하고 싶다면 ?
*** [IBM CLoud site](https://console.bluemix.net/catalog/services/machine-learning) 에서 Watson machine learning 서비스를 활용하시면 됩니다. ***